In [1]:
import configparser
import unittest
import datetime
import sys
import pandas as pd

# from tqdm import tqdm_notebook

sys.path.append("/home/sergey/drclinics/common")
from universal_connection import UniversalConnection, DBType

sys.path.append("/home/sergey/drclinics/reports")
# from report_utils import excel_report

import numpy as np
from log import log

In [55]:
import datetime

In [79]:
from dateutil.relativedelta import relativedelta

In [224]:
sql = """
select 	promo.id promo_id,
		prod.full_name product_name,
		--patient.id patient_id,
		--date(promo.start_ + 3 * interval '1 hour') date_start,
		--date(promo.end_ + 3 * interval '1 hour' - interval '1 year') date_start_end,
		case 
        	when date(promo.start_ + 3 * interval '1 hour') < '2017-10-01' then date(promo.end_ + 3 * interval '1 hour' - interval '1 year')
        	else date(promo.start_ + 3 * interval '1 hour')
       	end normal_start,
		--date_part('year', promo.start_ + 3 * interval '1 hour') start_year,
		--date_part('month', promo.start_ + 3 * interval '1 hour') start_month,
		date(promo.end_ + 3 * interval '1 hour') date_end,
		--date_part('year', promo.end_ + 3 * interval '1 hour') start_year,
		--date_part('month', promo.end_ + 3 * interval '1 hour') start_month,
		a.id app_id,
		date(a.finished + 3 * interval '1 hour') app_finished--,
		--date_part('year', a.finished + 3 * interval '1 hour') app_finished_year,
		--date_part('month', a.finished + 3 * interval '1 hour') app_finished_month
from promotion promo
inner join product_condition pc
   on promo.product_condition_id = pc.id
inner join product prod
   on prod.id = pc.product_id
----------------
inner join appointment a on promo.id = a.promotion_id 
where lower(prod.full_name) not like '%подписка%' 
and a.good
  and not (
    a.patient_id in (
        select patient_id
        from patient_categories pcat
        inner join reference rf
           on rf.id=pcat.reference_id
        where rf.code='TEST'
        )
    or lower(a.report_comment)='тест'
    )
and date(a.finished + 3 * interval '1 hour') is not null
--and date(promo.start_ + 3 * interval '1 hour') < '1990-01-01'
order by promo.id
"""

In [225]:
connection = UniversalConnection('../../../.credentials/telemed/prom.cfg', DBType.Postgres)
data = connection.query(sql)
connection.close()

2020-05-08 15:02:00 connect to postgres database using config file "../../../.credentials/telemed/prom.cfg"
2020-05-08 15:02:00 creating ssh tunnel to 172.16.100.19 as root...
2020-05-08 15:02:00 connect postgres using parameters:
                    database: telemed
				    user: norekhov
				    password: ***masked***
				    host: localhost
				    port: 34451
2020-05-08 15:02:00 @telemed: execute sql:
				    SET TIME ZONE 'Europe/Moscow'
				    None
2020-05-08 15:02:00 @telemed query:
                    select 	promo.id promo_id,
				    		prod.full_name product_name,
				    		--patient.id patient_id,
				    		--date(promo.start_ + 3 * interval '1 hour') date_start,
				    		--date(promo.end_ + 3 * interval '1 hour' - interval '1 year') date_start_end,
				    		case 
				            	when date(promo.start_ + 3 * interval '1 hour') < '2017-10-01' then date(promo.end_ + 3 * interval '1 hour' - interval '1 year')
				            	else date(promo.start_ + 3 * interval '1 hour')

In [226]:
df = pd.DataFrame(data).copy()

In [227]:
df.head()

,promo_id,product_name,normal_start,date_end,app_id,app_finished
0,32961,Капитал Здоровья. Индивидуальный,2018-02-20,2018-06-01,3929,2018-03-12
1,32961,Капитал Здоровья. Индивидуальный,2018-02-20,2018-06-01,4374,2018-03-27
2,32961,Капитал Здоровья. Индивидуальный,2018-02-20,2018-06-01,4373,2018-03-27
3,32961,Капитал Здоровья. Индивидуальный,2018-02-20,2018-06-01,4868,2018-04-11
4,33234,Капитал Здоровья. Индивидуальный,2018-03-01,2018-06-01,12816,2018-05-03


In [228]:
#df['OrderPeriod'] = df.app_finished.apply(lambda x: x.strftime('%Y-%m'))
#df.head()

In [229]:
df.set_index('promo_id', inplace=True)

df['start'] = df.groupby(level=0)['normal_start'].min().apply(lambda x: x.strftime('%Y-%m'))
df.reset_index(inplace=True)

In [230]:
df['year-month-app'] = df.app_finished.apply(lambda x: x.strftime('%Y-%m'))

In [231]:
#a = datetime.datetime(int(df.loc[1]['start'].split('-')[0]), int(df.loc[1]['start'].split('-')[1]), 1)

In [232]:
#b = datetime.datetime(int(df.loc[1]['year-month-app'].split('-')[0]), int(df.loc[1]['year-month-app'].split('-')[1]), 1)

In [233]:
#r = relativedelta(b, a)

In [234]:
#r.months

In [235]:
df['cohort_period'] = ''
for i in range(df.shape[0]):
    a = datetime.datetime(int(df.loc[i]['start'].split('-')[0]), int(df.loc[i]['start'].split('-')[1]), 1)
    b = datetime.datetime(int(df.loc[i]['year-month-app'].split('-')[0]), int(df.loc[i]['year-month-app'].split('-')[1]), 1)
    r = relativedelta(b, a)
    
    #df['cohort_period'][i] = abs(r.months) + 1
    df['cohort_period'][i] = r.months + 1

In [238]:
df = df[df['cohort_period'] > 0]

In [239]:
df.cohort_period.unique()

array([2, 3, 1, 4, 8, 5, 9, 6, 12, 11, 10, 7], dtype=object)

In [240]:
df.head(20)

,promo_id,product_name,normal_start,date_end,app_id,app_finished,start,year-month-app,cohort_period
0,32961,Капитал Здоровья. Индивидуальный,2018-02-20,2018-06-01,3929,2018-03-12,2018-02,2018-03,2
1,32961,Капитал Здоровья. Индивидуальный,2018-02-20,2018-06-01,4374,2018-03-27,2018-02,2018-03,2
2,32961,Капитал Здоровья. Индивидуальный,2018-02-20,2018-06-01,4373,2018-03-27,2018-02,2018-03,2
3,32961,Капитал Здоровья. Индивидуальный,2018-02-20,2018-06-01,4868,2018-04-11,2018-02,2018-04,3
4,33234,Капитал Здоровья. Индивидуальный,2018-03-01,2018-06-01,12816,2018-05-03,2018-03,2018-05,3
5,33242,Эрго. Все врачи,2018-03-02,2018-06-30,3729,2018-03-05,2018-03,2018-03,1
6,33504,Капитал Здоровья. Семейный,2018-03-14,2018-06-01,14366,2018-06-07,2018-03,2018-06,4
7,33815,Капитал Здоровья. Индивидуальный,2018-03-15,2018-06-01,4368,2018-03-27,2018-03,2018-03,1
8,33829,Капитал Здоровья. Семейный,2018-03-16,2018-06-01,4039,2018-03-16,2018-03,2018-03,1
9,33829,Капитал Здоровья. Семейный,2018-03-16,2018-06-01,4038,2018-03-16,2018-03,2018-03,1


In [241]:
cohorts = df.groupby(['product_name', 'cohort_period'])['app_id'].count().reset_index()

In [242]:
cohorts.rename(columns={'app_id': 'consultations'
                        }, inplace=True)

In [243]:
#cohorts.set_index('product_name', inplace=True)

In [244]:
cohorts.head(20)

,product_name,cohort_period,consultations
0,VIP обследование для мужчин,2,1
1,VIP-обследование для женщин,1,7
2,VIP-обследование для женщин,2,12
3,VIP-обследование для женщин,3,22
4,VIP-обследование для женщин,4,22
5,VIP-обследование для женщин,5,11
6,VIP-обследование для женщин,6,10
7,VIP-обследование для женщин,7,9
8,VIP-обследование для женщин,8,2
9,VIP-обследование для женщин,9,5


In [245]:
cohorts.cohort_period.unique()

array([ 2,  1,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [246]:
periods = ['total', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [247]:
d = dict()

In [248]:
for i in range(cohorts.shape[0]):
    d[cohorts['product_name'][i]] = d.get(cohorts['product_name'][i], [0 for period in periods]) 
    d[cohorts['product_name'][i]][periods.index(cohorts['cohort_period'][i])] += cohorts['consultations'][i]
    d[cohorts['product_name'][i]][0] += cohorts['consultations'][i]

In [249]:
#cohorts['cohort_period'][1]

In [250]:
#abs(periods.index(cohorts['cohort_period'][12]))

In [251]:
report = pd.DataFrame.from_dict(d, orient='index').sort_index()

In [252]:
report.columns = periods

In [260]:
report = report.sort_values('total', ascending=False)

In [271]:
report.head(25)

,total,1,2,3,4,5,6,7,8,9,10,11,12
Телемедицина_ВЭБ,29404,143,21432,7829,0,0,0,0,0,0,0,0,0
РГС Телемедицина ФЛ,16300,1446,2475,2045,1899,1537,1351,1274,994,900,829,757,793
СБСЖ_Премьер_2018_06_16,11102,1006,1426,1248,1200,1057,932,916,958,662,594,578,525
Телемедицина в ДМС Ренессанс,10720,3587,1232,864,702,642,587,570,554,525,488,518,451
ДМС_Сбербанк,6634,555,657,848,667,980,1111,909,299,143,284,158,23
Добровольное страхование,6250,409,325,560,707,701,678,705,563,478,396,396,332
РГС Телемедицина ФЛ - семейная,3799,317,614,549,420,408,350,302,230,196,146,128,139
Телемедицина_ДТ_Факт,1606,1099,193,17,6,7,5,241,35,0,0,2,1
РГС Телемедицина ЮЛ,1364,291,365,123,101,99,51,63,64,46,55,52,54
Медицинская защита к ипотеке 1,1284,304,352,235,166,137,62,22,5,1,0,0,0


In [262]:
report_percent = report.copy()

In [263]:
for i in range(1, 13):
    report_percent[i] = round(report_percent[i] / report_percent['total'] * 100, 2)

In [265]:
# report_percent = report_percent.sort_values('total', ascending=False)

In [266]:
report_percent['total'] = 100

In [270]:
report_percent.head(25)

,total,1,2,3,4,5,6,7,8,9,10,11,12
Телемедицина_ВЭБ,100,0.49,72.89,26.63,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
РГС Телемедицина ФЛ,100,8.87,15.18,12.55,11.65,9.43,8.29,7.82,6.10,5.52,5.09,4.64,4.87
СБСЖ_Премьер_2018_06_16,100,9.06,12.84,11.24,10.81,9.52,8.39,8.25,8.63,5.96,5.35,5.21,4.73
Телемедицина в ДМС Ренессанс,100,33.46,11.49,8.06,6.55,5.99,5.48,5.32,5.17,4.90,4.55,4.83,4.21
ДМС_Сбербанк,100,8.37,9.90,12.78,10.05,14.77,16.75,13.70,4.51,2.16,4.28,2.38,0.35
Добровольное страхование,100,6.54,5.20,8.96,11.31,11.22,10.85,11.28,9.01,7.65,6.34,6.34,5.31
РГС Телемедицина ФЛ - семейная,100,8.34,16.16,14.45,11.06,10.74,9.21,7.95,6.05,5.16,3.84,3.37,3.66
Телемедицина_ДТ_Факт,100,68.43,12.02,1.06,0.37,0.44,0.31,15.01,2.18,0.00,0.00,0.12,0.06
РГС Телемедицина ЮЛ,100,21.33,26.76,9.02,7.40,7.26,3.74,4.62,4.69,3.37,4.03,3.81,3.96
Медицинская защита к ипотеке 1,100,23.68,27.41,18.30,12.93,10.67,4.83,1.71,0.39,0.08,0.00,0.00,0.00


In [268]:
file_name = 'products-months.xls'
writer = pd.ExcelWriter(file_name, engine='xlsxwriter')
report.to_excel(writer, sheet_name='Количество консультаций')
report_percent.to_excel(writer, sheet_name='Процентное соотношение')

writer.save()